In [46]:
import sys
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import shapely
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point
from typing import Tuple
#from sklearn.cluster import KMeans
from syspy.spatial.spatial import nearest, agglomerative_clustering, voronoi_diagram_dataframes, add_geometry_coordinates


In [47]:
zones = gpd.read_file('../../inputs/base/zones/zones.geojson')
zones = zones.set_index('index')

In [48]:
def round_geom(g, precision):
    return shapely.geometry.Point([round(c, precision) for c in g.coords[0]])

#zones['geometry'] = zones['geometry'].apply(lambda g: round_geom(g, 6))
node_lists = zones.reset_index().groupby('geometry')['index'].agg(list).values

In [49]:
nodes = [l[0] for l in node_lists]

In [50]:
zones2 = zones.loc[nodes]

In [53]:
convex_hull = zones.unary_union.convex_hull.buffer(1e-2)
voronoi = voronoi_diagram_dataframes(zones2['geometry'])
voronoi = gpd.GeoDataFrame(voronoi[0],crs=4326)
#voronoi = pd.merge(voronoi,zones[['production','attraction']],left_index=True,right_index=True)
voronoi = voronoi.clip(convex_hull)
#voronoi.to_file(output_folder + 'zones.geojson',driver='GeoJSON')


/home/boivin/.local/share/virtualenvs/quetzal-77-onnKO/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [54]:
voronoi

,geometry
58,"POLYGON ((-0.81991 51.58017, -0.85408 51.57889..."
237,"POLYGON ((-1.08012 51.64276, -1.08277 51.63091..."
582,"POLYGON ((-0.57366 51.70041, -0.53484 51.68333..."
239,"POLYGON ((-1.02079 51.67493, -0.97880 51.63987..."
173,"POLYGON ((-1.02520 51.68302, -1.02079 51.67493..."
...,...
271,"POLYGON ((-2.49074 53.36502, -2.35890 52.86722..."
269,"POLYGON ((-1.70322 52.85199, -1.77469 52.74898..."
264,"POLYGON ((-1.27671 52.91045, -1.35480 52.81158..."
590,"POLYGON ((-1.94338 53.80335, -2.16148 53.54472..."


In [58]:
index_dict = nearest(voronoi,zones2).set_index('ix_one')['ix_many'].to_dict()
voronoi.index = voronoi.index.map(index_dict.get)

In [63]:
zones['geometry'] = zones['geometry'].apply(lambda g: g.buffer(1e-3))

In [64]:
zones = zones.merge(voronoi,left_index=True,right_index=True,how='left')

In [65]:
zones['geometry'] = zones['geometry_y'].fillna(zones['geometry_x'])

In [66]:
zones = zones.drop(columns=['geometry_x','geometry_y'])

In [67]:
gpd.GeoDataFrame(zones).to_file('../../inputs/base/zones/voronoi.geojson',driver='GeoJSON')